# Research on Borrower Reliability

Client: The credit department of the bank. The task is to investigate whether the marital status and the number of children of a client influence the timely repayment of a loan. The input data from the bank includes statistics on the solvency of clients.

The research findings will be taken into account in the construction of a **credit scoring** model — a specialized system that assesses the ability of a potential borrower to repay a loan to the bank.

## Step 1. Open the data file and review the general information

In [12]:
import pandas as pd
from IPython.display import display

data = pd.read_csv('data.csv')
display(data.head())
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<a id='general_info'>**Conclusion**</a>

The table consists of 12 columns and 21,525 rows. Each row in the table represents data about a client and their loan.

According to the documentation for the data:
* `children` — the number of children in the family;
* `days_employed` — total work experience in days;
* `dob_years` — the client's age in years;
* `education` — the client's level of education;
* `education_id` — the identifier of the education level;
* `family_status` — the client's marital status;
* `family_status_id` — the identifier of the marital status;
* `gender` — the client's gender;
* `income_type` — the type of employment;
* `debt` — whether the client had a debt on the loan repayment;
* `total_income` — monthly income;
* `purpose` — the purpose of obtaining the loan.

There are missing values in the `total_income` and `days_employed` columns. Also, the data type in these columns is `float`, although based on the nature of the data (number of days and income), it should be `int`. The data types of other columns reflect reality. In the `days_employed` column, there are many negative values, which is also an artifact.

## Step 2. Data Preprocessing

### Handling Missing Values

First, let's calculate the number of missing values in the table.

In [13]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

As we can see, the number of missing values in the `days_employed` and `total_income` columns is the same, and the missing values occur for the same clients. Most likely, the data source simply did not have information about the work experience and income for these clients. The `days_employed` column does not affect the hypothesis testing and the final result, so we can either delete it or fill the missing values with an arbitrary value, such as zero.

In [14]:
data['days_employed'] = data['days_employed'].fillna(0)

But missing values in `total_income` can distort the final result. The best option to address these gaps would be to clarify the information with the developer who provided us with this data, but we do not have that option. Therefore, to fill in the missing values, we will calculate the median for each group based on the employment type and assign the corresponding values for each client based on their `income_type`.

In [15]:
data['total_income'] = data.groupby(['income_type'])['total_income'].transform(
    lambda income: income.fillna(income.median())
)

Let's make sure that there are no more missing values in the table.

In [16]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Data Type Conversion

As we established earlier, the columns `total_income` and `days_employed` are recognized as `float`. This implies that there are decimal values in these columns, which contradicts the nature of the data. We cannot precisely know the reason for this type, but it is likely a result of some division.

For instance, the following calculations might have occurred for `days_employed`:
`secs_employed = end_of_work - start_of_work`, where `end_of_work` and `start_of_work` are the end and start times of work in **Unix time** format.
`days_employed = secs_employed / 60 / 60 / 24` - conversion of seconds to days.

Therefore, for more convenient further processing, let's convert the `total_income` and `days_employed` data to integers using the `astype()` method.

In [17]:
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')

Let's check the result obtained by calling the `info()` method.

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


It is evident that only integer values are now retained in the `total_income` and `days_employed` columns.

When we examined the [general information](#general_info) about the table, we noticed that there are many negative values in the `days_employed` column. We can assume that they arose due to the incorrect order of operands during subtraction (`start_of_work - end_of_work`). To ensure that the data reflects reality, let's convert the negative values to positive using the `abs()` function.

In [19]:
data['days_employed'] = abs(data['days_employed'])

### Handling Duplicates

In [20]:
data.duplicated().sum()

54

In the table, we have 54 duplicates. It is unlikely that we would encounter different individuals with exactly the same values in all columns. Therefore, we can assume that the duplicates occurred due to a database glitch, and we can get rid of them using the `drop_duplicates()` method. Let's remove the obvious duplicates by dropping the old indices and generating new ones:

In [21]:
data = data.drop_duplicates().reset_index(drop=True)

Let's make sure that we have completely eliminated the obvious duplicates.

In [22]:
data.duplicated().sum()

0

Now let's try to find implicit duplicates in each column where their presence is likely:

In [23]:
print(
    data['education'].value_counts(),
    data['family_status'].value_counts(),
    data['income_type'].value_counts(),
    data['purpose'].value_counts(),
    sep='\n\n'
)

education
среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: count, dtype: int64

family_status
женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: count, dtype: int64

income_type
сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: count, dtype: int64

purpose
свадьба                                   793
на проведение свадьбы                  

It is evident that there are many implicit duplicates in the `education` column, differing in case. In the `purpose` column, the same loan purposes are recorded in different forms. In all other columns, the values are unique.
To eliminate duplicates in `education`, we will convert all values to lowercase using the `str.lower()` method. Eliminating duplicates in `purpose` requires a more complex process - [**lemmatization**](#lemmatization).

In [24]:
data['education'] = data['education'].str.lower()

Let's check whether there are any duplicates in the table after our transformations.

In [25]:
data.duplicated().sum()

17

Using the `drop_duplicates()` method, let's get rid of the duplicates. Afterward, we will verify if we have successfully removed all of them.

In [26]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

There are no duplicates in the table now. We can proceed to the next step - lemmatization.

### <a id='lemmatization'>Lemmatization</a>

Let's output a list of unique values present in the `purpose` column.

In [27]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

There are a total of 38 different values, but upon closer inspection, many share a common purpose with different phrasing.

For example, the entire list:
* purchase of housing (покупка жилья)
* purchase of housing for a family (покупка жилья для семьи)
* purchase of real estate (покупка недвижимости)
* purchase of residential real estate (покупка жилой недвижимости)
* construction of own real estate (строительство собственной недвижимости)
* construction of real estate (строительство недвижимости)

describes a single loan purpose – for real estate.

Therefore, we can categorize the purposes into a few groups:
* Commercial (коммерция)
* Real estate (недвижимость)
* Wedding (свадьба)
* Car (автомобиль)
* Education (образование)

To move from various descriptions of the purpose to a consistent one, we will use the lemmatization process.

To begin, let's import the `pymystem3` library for lemmatization.

In [28]:
from pymystem3 import Mystem
m = Mystem()

Let's lemmatize the words for each unique purpose:

In [29]:
for unique_purpose in data['purpose'].unique():
    purpose_lemmas = m.lemmatize(unique_purpose)
    print(purpose_lemmas)

['покупка', ' ', 'жилье', '\n']
['приобретение', ' ', 'автомобиль', '\n']
['дополнительный', ' ', 'образование', '\n']
['сыграть', ' ', 'свадьба', '\n']
['операция', ' ', 'с', ' ', 'жилье', '\n']
['образование', '\n']
['на', ' ', 'проведение', ' ', 'свадьба', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n']
['покупка', ' ', 'недвижимость', '\n']
['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n']
['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n']
['недвижимость', '\n']
['строительство', ' ', 'недвижимость', '\n']
['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n']
['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n']
['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n']
['жилье', '\n']
['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n']
['автомобиль', '\n']
['заниматься', ' ', 'образование'

Now we can associate the presence of a specific lemma with categories. Let's create a dictionary where the key is the lemma, and the value is our category.

In [30]:
purpose_categories = {
        'коммерческий': 'коммерция',
        'жилье': 'недвижимость',
        'недвижимость': 'недвижимость',
        'свадьба': 'свадьба',
        'автомобиль': 'автомобиль',
        'образование': 'образование'
    }

Let's write a function that takes a `purpose` value and returns the corresponding category from our dictionary. We will then apply this function to the entire `purpose` column using the `apply` method, and store the function's output in a new column called `purpose_category`.

In [31]:
def get_purpose_category(purpose):
    lemmas = m.lemmatize(purpose)
    for lemma, category in purpose_categories.items():
        if lemma in lemmas:
            return category

data['purpose_category'] = data['purpose'].apply(get_purpose_category)

**Conclusion**
Lemmatization has allowed us to reduce the list of 38 loan purposes to 5. In the future, this will help us in hypothesis testing -
**How do different loan purposes affect their timely repayment?**

One of the hypotheses we need to test sounds like this - **Is there a correlation between having children and repaying the loan on time?** This means we need to group our data based on the values in the `children` column.

Let's categorize clients into two groups:
* if the `children` value is greater than zero, it means **there are children**
* if the `children` value is zero, the family is **childless**

We will define the rules for classifying clients as a function, where the input is the number of children, and the output is the client's category.

In [32]:
def family_children_type(children_amount):
    if children_amount == 0:
        return 'без детей'
    else:
        return 'есть дети'

Let's test the function for each rule:

In [33]:
print(family_children_type(0))
print(family_children_type(2))

без детей
есть дети


The function works correctly. Now, let's create a separate column for the category, and in its cells, input the values returned by the function. We will use the `apply()` method for this.

In [34]:
data['children_category'] = data['children'].apply(family_children_type)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,есть дети
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,есть дети
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,без детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,есть дети
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,без детей


The data is categorized based on the presence of children for clients, and we can also categorize the data based on monthly income. Since we don't have reference points to divide the data into poor, middle, and rich classes, we will use percentiles to divide the data into three groups. Let's find the necessary points:

In [35]:
data['total_income'].describe(percentiles=[0.33, 0.66])

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
33%      1.185140e+05
50%      1.425940e+05
66%      1.723570e+05
max      2.265604e+06
Name: total_income, dtype: float64

It's evident how we can divide the data into three categories using percentiles:
* **Q1** - `total_income < 118,514`
* **Q2** - `118,514 <= total_income < 172,357`
* **Q3** - `total_income >= 172,357`

Now we can write a function that will classify clients based on `total_income`. The function takes the amount of monthly income as input and returns the category.

In [36]:
def income_category(income):
    if income < 118514:
        return 'Q1'
    elif 118514 <= income < 172357:
        return 'Q2'
    else:
        return 'Q3'

Let's check if the function works correctly:

In [37]:
print(income_category(250))
print(income_category(125550))
print(income_category(345550))

Q1
Q2
Q3


The function works correctly. Now we can apply it to the entire dataset:

In [38]:
data['income_category'] = data['total_income'].apply(income_category)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,есть дети,Q3
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,есть дети,Q1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,без детей,Q2
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,есть дети,Q3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,без детей,Q2


**Conclusion**
We have successfully classified the data based on the presence of children for clients, dividing the data into two groups - **"have children"** and **"childless"**. We also classified clients based on monthly income, dividing them into three groups: **Q1** - the bottom 33% of earners, **Q3** - the top 33% of earners, and **Q2** - all others. These data manipulations are necessary to make statistically significant conclusions when testing hypotheses.

## Step 3. Answer the questions

**Is there a correlation between having children and timely loan repayment?**

To answer this question, we need to group the data by `children_category`. For each category, calculate the percentage of clients who had a debt on the loan.

In [39]:
data_by_children = data.groupby('children_category')
print(data_by_children['debt'].sum() / data_by_children['children'].count())

children_category
без детей    0.075438
есть дети    0.092082
dtype: float64


**Conclusion**
Clients without children repay loans approximately 20% more often.

**Is there a correlation between marital status and timely loan repayment?**

In [40]:
data_by_family_status = data.groupby('family_status')
print(data['debt'].sum() / data['family_status'].count())
print(data_by_family_status['debt'].sum() / data_by_family_status['family_status'].count())

0.08115036822970076
family_status
Не женат / не замужем    0.097509
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093471
женат / замужем          0.075452
dtype: float64


**Conclusion**
The percentage of non-repayment varies depending on marital status within the range of 6.56-9.75%. According to the data, the most reliable clients are widows and widowers, while the least reliable are those not married.

**Is there a correlation between income level and timely loan repayment?**

In [41]:
data_by_income = data.groupby('income_category')
print(data_by_income['debt'].sum() / data_by_income['income_category'].count())

income_category
Q1    0.081085
Q2    0.088431
Q3    0.074212
dtype: float64


**Conclusion**
There is a difference, but it is not as significant as in the previous points. However, we can say that the most responsible borrowers are individuals with a high income level.

**How do different loan purposes affect their timely repayment?**

In [42]:
data_by_purpose = data.groupby('purpose_category')
print(data_by_purpose['debt'].sum() / data_by_purpose['purpose'].count())

purpose_category
автомобиль      0.093590
коммерция       0.075515
недвижимость    0.071895
образование     0.092200
свадьба         0.080034
dtype: float64


**Conclusion**
Loan purposes do affect their repayment. Clients taking out loans for education, understandably, have more debt, as they likely do not have a stable income yet. Also, loans for consumer purposes (car, wedding) indicate a lower income level and a worse ability to repay debts, while investment purposes (commercial and real estate) correlate with better repayments.

## Step 4. General Conclusion

We have tested 4 hypotheses and found the following:
1. Clients without children repay loans more often than those with children.
2. Clients who are not married are the least likely to repay loans, while widows and widowers have proven to be reliable borrowers.
3. As expected, individuals with a high monthly income are the most responsible in terms of loan repayment.
4. Investment purposes (commercial and real estate) correlate with better repayments.

The percentage of problematic borrowers varies within the range of 6.6-9.8% depending on the group, with an overall percentage of 8% across all data. It seems that we cannot draw any conclusions just by looking at these numbers; at the very least, we need to understand what to compare them with. To claim any dependence between repayments and a specific group, a statistical analysis is needed, of which I currently have limited knowledge.